In [26]:
import os
import pandas as pd
import numpy as np
from math import sqrt
from tqdm import tqdm_notebook as tqdm
import scipy

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import matplotlib.font_manager as fm
import matplotlib
import itertools

In [27]:
fm.get_fontconfig_fonts()
font_location = 'C:\\WINDOWS\\Fonts\\malgun.ttf' # For Windows
font_name = fm.FontProperties(fname=font_location).get_name()
matplotlib.rc('font', family=font_name)

C:\Users\bongkyun\AppData\Local\Temp\ipykernel_13900\2582751676.py:1: MatplotlibDeprecationWarning: 
The get_fontconfig_fonts function was deprecated in Matplotlib 3.5 and will be removed two minor releases later.
  fm.get_fontconfig_fonts()


In [28]:
FILES_DIR = '../data/a1/'
log_df = pd.read_csv(FILES_DIR + 'log_csv.csv')

In [29]:
# null값들을 un_** 으로 채워서 확인해보기
log_drop_null = log_df
log_drop_null['geoip_city_name'].fillna('un_city_name', inplace = True)
log_drop_null['category3'].fillna('un_category', inplace = True)
log_drop_null['uid'].fillna('un_name', inplace = True)
log_drop_null.dropna(axis = 0, inplace = True)
print(log_drop_null.isnull().sum().sort_values(ascending=False)) # null값 퍼센트로 표현


# log_drop_null['rating'] = log_drop_null['action_type']
# re_name = {'rating' : {'View':1, 'Highlight':2, 'Basket':4, 'Copy':3, 'SINF':0}}
# log_drop_null = log_drop_null.replace(re_name)
# log_drop_null = log_drop_null[['uid', 'category2', 'rating']]
# log_drop_null                                        

Unnamed: 0         0
uid                0
action_type        0
category1          0
category2          0
category3          0
collect_time       0
geoip_city_name    0
useragent_os       0
viewrate           0
viewtime           0
dtype: int64


In [30]:
log_drop_null['rating'] = log_drop_null['action_type']
re_name = {'rating' : {'View':1, 'Highlight':2, 'Basket':4, 'Copy':3, 'SINF':0}}
log_drop_null = log_drop_null.replace(re_name)
log_drop_null

,Unnamed: 0,uid,action_type,category1,category2,category3,collect_time,geoip_city_name,useragent_os,viewrate,viewtime,rating
0,0,avpKyWMn/xEADNlb,View,주방용품,그릇·홈세트,식판·나눔접시,2022-12-29 17:29:02,Seongnam-si,Windows,100.0,80033,1
1,1,0+5vEmGkOuAAB5j2,Basket,패브릭,매트,주방·다용도매트,2022-12-29 12:56:38,Yongin-si,Windows,0.0,0,4
2,2,r8DXy2OtSl8AAojG,View,패브릭,커튼·부자재,암막커튼,2022-12-29 17:11:30,Seoul,iOS,100.0,947,1
3,3,dC9Fl2Ly9lcADrTn,Highlight,데코·식물,플라워·식물,화병·화분,2022-12-29 17:54:30,Gwangmyeong-si,Windows,0.0,0,2
4,4,09gKcGLg0I8ACyQy,View,가구,진열장·책장,책장,2022-12-29 18:45:05,Jongno-gu,Mac OS X,100.0,1238,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5659536,5659536,PUpxWWMWdcIACR2B,View,주방용품,칼·커팅기구,채칼·다지기·강판,2022-12-28 20:01:30,Seoul,Android,100.0,13893,1
5659537,5659537,PVMstmNg4I4ADq9R,Basket,가구,진열장·책장,책장,2022-12-28 14:30:09,Jinju,Windows,0.0,0,4
5659538,5659538,diRXdmN49dUABrVo,View,가구,테이블·식탁·책상,식탁,2022-12-29 12:45:13,Gunpo,Android,100.0,47217,1
5659539,5659539,dusFzWJvp5cADhZW,View,가전·디지털,계절가전,전기히터·온풍기,2022-12-28 11:07:30,Seoul,Android,100.0,54030,1


In [31]:
log = log_drop_null.drop_duplicates()
log

,Unnamed: 0,uid,action_type,category1,category2,category3,collect_time,geoip_city_name,useragent_os,viewrate,viewtime,rating
0,0,avpKyWMn/xEADNlb,View,주방용품,그릇·홈세트,식판·나눔접시,2022-12-29 17:29:02,Seongnam-si,Windows,100.0,80033,1
1,1,0+5vEmGkOuAAB5j2,Basket,패브릭,매트,주방·다용도매트,2022-12-29 12:56:38,Yongin-si,Windows,0.0,0,4
2,2,r8DXy2OtSl8AAojG,View,패브릭,커튼·부자재,암막커튼,2022-12-29 17:11:30,Seoul,iOS,100.0,947,1
3,3,dC9Fl2Ly9lcADrTn,Highlight,데코·식물,플라워·식물,화병·화분,2022-12-29 17:54:30,Gwangmyeong-si,Windows,0.0,0,2
4,4,09gKcGLg0I8ACyQy,View,가구,진열장·책장,책장,2022-12-29 18:45:05,Jongno-gu,Mac OS X,100.0,1238,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5659536,5659536,PUpxWWMWdcIACR2B,View,주방용품,칼·커팅기구,채칼·다지기·강판,2022-12-28 20:01:30,Seoul,Android,100.0,13893,1
5659537,5659537,PVMstmNg4I4ADq9R,Basket,가구,진열장·책장,책장,2022-12-28 14:30:09,Jinju,Windows,0.0,0,4
5659538,5659538,diRXdmN49dUABrVo,View,가구,테이블·식탁·책상,식탁,2022-12-29 12:45:13,Gunpo,Android,100.0,47217,1
5659539,5659539,dusFzWJvp5cADhZW,View,가전·디지털,계절가전,전기히터·온풍기,2022-12-28 11:07:30,Seoul,Android,100.0,54030,1


In [60]:
log_sample = log.sample(frac=0.01, random_state=42)
log_sample = log_sample[~log_sample.index.duplicated(keep='first')]
log_sample = log_sample[:20000]
log_sample

,Unnamed: 0,uid,action_type,category1,category2,category3,collect_time,geoip_city_name,useragent_os,viewrate,viewtime,rating
3595088,3595088,bgojimOqhQEACbYY,View,가전·디지털,휴대폰,휴대폰,2022-12-27 14:39:16,Seoul,Android,100.00,3006,1
966238,966238,0zPC5WOId8YABX08,Copy,가전·디지털,계절가전,전기요·온수매트,2022-12-26 10:16:23,Yeongdeungpo-gu,Mac OS X,0.00,0,3
2237030,2237030,t2umo2IVzewAAJiY,Highlight,패브릭,홈패브릭,홈웨어,2022-12-26 13:08:35,Daejeon,Windows,0.00,0,2
868149,868149,AeM3zWOgVs4ABfZU,View,패브릭,러그·카페트,극세사·단모러그,2022-12-24 03:11:27,Yongin-si,iOS,100.00,3906,1
4828400,4828400,cJLHnmOuWO4AA50D,View,노하우,생활정보,un_category,2022-12-30 13:39:20,Seongdong-gu,Mac OS X,100.00,44363,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1817834,1817834,02iMZGH5STIAB+MS,View,가구,테이블·식탁·책상,책상,2022-12-23 22:44:51,Gangnam-gu,Windows,100.00,18842,1
1621583,1621583,t2P6DWD5Kb0AAcOe,View,노하우,청소,un_category,2022-12-22 10:28:57,Suwon,Windows,100.00,38150,1
4485827,4485827,0rcYrGOYQY4ABVs6,View,패브릭,블라인드·롤스크린,블라인드,2022-12-22 12:42:22,Yongin-si,Windows,100.00,153899,1
1296432,1296432,0zPC5WFDO8YAA+Vg,View,패브릭,매트,현관·발매트,2022-12-27 21:51:36,Yeongdeungpo-gu,Windows,99.92,6395,1


In [61]:
log_sample.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20000 entries, 3595088 to 4419407
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       20000 non-null  int64  
 1   uid              20000 non-null  object 
 2   action_type      20000 non-null  object 
 3   category1        20000 non-null  object 
 4   category2        20000 non-null  object 
 5   category3        20000 non-null  object 
 6   collect_time     20000 non-null  object 
 7   geoip_city_name  20000 non-null  object 
 8   useragent_os     20000 non-null  object 
 9   viewrate         20000 non-null  float64
 10  viewtime         20000 non-null  int64  
 11  rating           20000 non-null  int64  
dtypes: float64(1), int64(3), object(8)
memory usage: 2.0+ MB


In [62]:
train_df, test_df = train_test_split(log_sample, test_size=0.2, random_state=42)

print(train_df.shape)
print(test_df.shape)

(16000, 12)
(4000, 12)


In [63]:
category1 = [(list(set(x['uid'].values))[0], '/'.join(x['category1'].values)) for index, x in log_sample.groupby('uid')]
category1_df = pd.DataFrame(data=category1, columns=['uid', 'category1'])
category1_df = category1_df.set_index('uid')


category2 = [(list(set(x['uid'].values))[0], '/'.join(x['category2'].values)) for index, x in log_sample.groupby('uid')]
category2_df = pd.DataFrame(data=category2, columns=['uid', 'category2'])
category2_df = category2_df.set_index('uid')


countries = [(list(set(x['uid'].values))[0], ','.join(x['geoip_city_name'].values)) for index, x in log_sample.groupby('uid')]
countries_df = pd.DataFrame(data=countries, columns=['uid', 'country'])
countries_df = countries_df.set_index('uid')


os = [(list(set(x['uid'].values))[0], ','.join(x['useragent_os'].values)) for index, x in log_sample.groupby('uid')]
os_df = pd.DataFrame(data=os, columns=['uid', 'OS'])
os_df = os_df.set_index('uid')



log_total = pd.concat([category1_df, category2_df, countries_df, os_df], axis=1)

print(log_total.shape)
print(log_total.head())

(16452, 4)
                 category1 category2       country       OS
uid                                                        
0+0hA2OFU6AADGdD    가전·디지털   생활·건강가전  un_city_name  Windows
0+157mLfUpoABHkY        가구   거실장·TV장     Goyang-si  Windows
0+2w2mGmCYIADF71     데코·식물    플라워·식물        Jeonju  Android
0+38jmECs5IACrzg    가전·디지털      저장장치     Gwanak-gu  Windows
0+4qI2N6x/kABcma        가구        소파  un_city_name  Windows


In [64]:
dummy_cat1_df = log_total['category1'].str.get_dummies(sep='/')
dummy_cat1_df

,3M,DELI,DF동서가구,Decor Plant,Ev Aletleri Dijital,GALON (갈롱),HAILY(헤일리),HICKIES,Herman Miller,Home Appliances Digital,...,한일카페트,헤이몰리,헤즈로,현대가구,호시재,홈바내쓰,홈앤하우스,홈인홈,휴도,히츠키코보
uid,,,,,,,,,,,,,,,,,,,,,
0+0hA2OFU6AADGdD,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0+157mLfUpoABHkY,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0+2w2mGmCYIADF71,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0+38jmECs5IACrzg,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0+4qI2N6x/kABcma,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
yr5IcGN6VegAAcpi,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ywVMxGOqSAIADMkm,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ywpjRWMglBcAC53F,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [65]:
dummy_cat2_df = log_total['category2'].str.get_dummies(sep='/')
dummy_cat2_df

,"Bowl, home set",DIY&리폼,DIY·취미·공예,LED 등,Mattress Topper,O!PLATING,O!STYLING,O!SelectShop,Sofa,TV,...,형광등·조명부속품,홈갤러리,홈데코,홈케어,홈패브릭,화장대·콘솔,화장대·테이블정리,화장지·물티슈,후크·수납걸이,휴대폰
uid,,,,,,,,,,,,,,,,,,,,,
0+0hA2OFU6AADGdD,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0+157mLfUpoABHkY,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0+2w2mGmCYIADF71,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0+38jmECs5IACrzg,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0+4qI2N6x/kABcma,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
yr5IcGN6VegAAcpi,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ywVMxGOqSAIADMkm,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ywpjRWMglBcAC53F,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [66]:
dummy_country_df = log_total['country'].str.get_dummies(sep=',')
dummy_country_df

,Acapulco de Juárez,Adana,Amsterdam,Andong,Angeles City,Annaka,Ansan-si,Anseong,Anyang-si,Asan,...,Yesan-gun,Yokohama,Yongin,Yongin-si,Yongsan-gu,Yunlin,Yuseong,Yuseong-gu,Zurich,un_city_name
uid,,,,,,,,,,,,,,,,,,,,,
0+0hA2OFU6AADGdD,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
0+157mLfUpoABHkY,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0+2w2mGmCYIADF71,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0+38jmECs5IACrzg,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0+4qI2N6x/kABcma,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
yr5IcGN6VegAAcpi,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ywVMxGOqSAIADMkm,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
ywpjRWMglBcAC53F,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
dummy_os_df = log_total['OS'].str.get_dummies(sep=',')
dummy_os_df

,Android,CentOS,Chrome OS,Linux,Mac OS X,Tizen,Ubuntu,Windows,iOS
uid,,,,,,,,,
0+0hA2OFU6AADGdD,0,0,0,0,0,0,0,1,0
0+157mLfUpoABHkY,0,0,0,0,0,0,0,1,0
0+2w2mGmCYIADF71,1,0,0,0,0,0,0,0,0
0+38jmECs5IACrzg,0,0,0,0,0,0,0,1,0
0+4qI2N6x/kABcma,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...
yr5IcGN6VegAAcpi,0,0,0,0,0,0,0,1,0
ywVMxGOqSAIADMkm,0,0,0,0,0,0,0,1,0
ywpjRWMglBcAC53F,0,0,0,0,0,0,0,1,0


In [68]:
X_train = pd.concat([pd.get_dummies(train_df['uid'], prefix='uid'),
           train_df['uid'].apply(lambda x: dummy_cat1_df.loc[x]),
           train_df['uid'].apply(lambda x: dummy_cat2_df.loc[x]),
           train_df['uid'].apply(lambda x: dummy_country_df.loc[x]),
           train_df['uid'].apply(lambda x: dummy_os_df.loc[x])], axis=1)

In [69]:
X_train

,uid_0+0hA2OFU6AADGdD,uid_0+157mLfUpoABHkY,uid_0+2w2mGmCYIADF71,uid_0+38jmECs5IACrzg,uid_0+4qI2N6x/kABcma,uid_0+57MmOtEeMACzd5,uid_0+5K/GOV/9AAAnSC,uid_0+6h2WOH7o4ACCXz,uid_0+6jh2AbcqEADrzv,uid_0+6ssWN2ZyEACYA0,...,un_city_name,Android,CentOS,Chrome OS,Linux,Mac OS X,Tizen,Ubuntu,Windows,iOS
4300594,False,False,False,False,False,False,False,False,False,False,...,0,0,0,0,0,1,0,0,0,0
2226,False,False,False,False,False,False,False,False,False,False,...,1,0,0,0,0,0,0,0,1,0
2980536,False,False,False,False,False,False,False,False,False,False,...,0,0,0,0,0,0,0,0,1,0
5503997,False,False,False,False,False,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,1
4899806,False,False,False,False,False,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1698316,False,False,False,False,False,False,False,False,False,False,...,0,0,0,0,0,1,0,0,0,0
3401689,False,False,False,False,False,False,False,False,False,False,...,0,1,0,0,0,0,0,0,0,0
227685,False,False,False,False,False,False,False,False,False,False,...,0,0,0,0,0,0,0,0,1,0
4698215,False,False,False,False,False,False,False,False,False,False,...,0,0,0,0,0,0,0,0,1,0


In [55]:
y_train = train_df['rating']
print(X_train.shape)
print(y_train.shape)

(10000, 10050)
(10000,)


In [56]:
X_train_sparse = scipy.sparse.csr_matrix(X_train.values)

TypeError: no supported conversion for types: (dtype('O'),)

In [ ]:
X_train_sparse

In [ ]:
def log_loss(pred, y):
    return np.log(np.exp(-pred * y) + 1.0)


# Update gradients
def sgd(X, y, n_samples, n_features,
                w0, w, v, n_factors, learning_rate, reg_w, reg_v):
    data = X.data
    indptr = X.indptr
    indices = X.indices
    loss = 0.0

    for i in range(n_samples):
        pred, summed = predict(X, w0, w, v, n_factors, i)
        
        # calculate loss and its gradient
        loss += log_loss(pred, y[i])
        loss_gradient = -y[i] / (np.exp(y[i] * pred) + 1.0)
    
        # update bias/intercept term
        w0 -= learning_rate * loss_gradient

        # update weight
        for index in range(indptr[i], indptr[i + 1]):
            feature = indices[index]
            w[feature] -= learning_rate * (loss_gradient * data[index] + 2 * reg_w * w[feature])

        # update factor
        for factor in range(n_factors):
            for index in range(indptr[i], indptr[i + 1]):
                feature = indices[index]
                term = summed[factor] - v[factor, feature] * data[index]
                v_gradient = loss_gradient * data[index] * term
                v[factor, feature] -= learning_rate * (v_gradient + 2 * reg_v * v[factor, feature])
    
    loss /= n_samples
    return loss




def predict(X, w0, w, v, n_factors, i):
    data = X.data
    indptr = X.indptr
    indices = X.indices
    """predicting a single instance"""
    summed = np.zeros(n_factors)
    summed_squared = np.zeros(n_factors)

    # linear output w * x
    pred = w0
    for index in range(indptr[i], indptr[i + 1]):
        feature = indices[index]
        pred += w[feature] * data[index]

    # factor output
    for factor in range(n_factors):
        for index in range(indptr[i], indptr[i + 1]):
            feature = indices[index]
            term = v[factor, feature] * data[index]
            summed[factor] += term
            summed_squared[factor] += term * term

        pred += 0.5 * (summed[factor] * summed[factor] - summed_squared[factor])

    # gradient update할 때, summed는 독립이므로 re-use 가능
    return pred, summed




# Train Factorization Machine
# X -> sparse csr_matrix, y -> label
def fit(X, y, config):
    epochs = config['num_epochs']
    num_factors = config['num_factors']
    learning_rate = config['learning_rate']
    reg_weights = config['reg_weights']
    reg_features = config['reg_features']

    num_samples, num_features = X.shape
    weights = np.zeros(num_features) # -> w
    global_bias = 0.0 # -> w0
    
    # latent factors for all features -> v
    feature_factors = np.random.normal(size = (num_factors, num_features))

    epoch_loss = []
    for epoch in range(epochs):
        loss = sgd(X, y, num_samples, num_features,
                            global_bias, weights,
                            feature_factors, num_factors,
                            learning_rate, reg_weights, reg_features)
        print(f'[epoch: {epoch+1}], loss: {loss}')

        epoch_loss.append(loss)
      
    return epoch_loss


In [ ]:
config = {
    "num_epochs": 10,
    "num_factors": 10,
    "learning_rate": 0.1,
    "reg_weights": 0.01,
    "reg_features": 0.01
}

In [ ]:
epoch_loss = fit(X_train_sparse, y_train.values, config)

In [ ]:
plt.plot(epoch_loss)
plt.title('Loss per epoch')
plt.show()